# 🌳 PyKwant Tutorial: Binomial Trees & American Options

This notebook introduces `pykwant.trees`, a module implementing Lattice methods (Binomial Trees) for option pricing.

While Black-Scholes is perfect for **European Options** (exercise only at expiry), it cannot handle **American Options** (exercise anytime). For these, we need numerical methods that step backwards through time to check for optimal early exercise.

We implement the **Cox-Ross-Rubinstein (CRR)** model in a functional style.

## 1. Setup and Market Data

We define a standard market environment: Spot 100, Volatility 20%, Risk-Free Rate 5%.

In [1]:
import math
from datetime import date

from pykwant import dates, equity, instruments, trees

# Market Data
val_date = date(2025, 1, 1)
spot_price = 100.0
volatility = 0.20


# Flat 5% Curve
def flat_curve(d: date) -> float:
    t = dates.act_365(val_date, d)
    return math.exp(-0.05 * t)


# Helper for comparison
def print_comparison(label, bs_price, tree_price):
    diff = tree_price - bs_price
    print(
        f"{label:<20} | BS (Euro): {bs_price:7.4f} | Tree (Amer): {tree_price:7.4f} | "
        f"Diff: {diff:7.4f}"
    )

## 2. Call Options: American vs European

Theory states that for a non-dividend paying stock, it is never optimal to exercise a **Call Option** early. Therefore, the price of an American Call should equal the European Call.

Let's verify this using our Tree (for American) and Black-Scholes (for European).

In [2]:
expiry = date(2026, 1, 1)
strike = instruments.Money(100.0)

# Define Instruments
eur_call = instruments.EuropeanOption("TEST", strike, expiry, "call")
amer_call = instruments.AmericanOption("TEST", strike, expiry, "call")

# Price
# Note: We use a high number of steps (200) for better convergence
bs_price = equity.black_scholes_price(eur_call, spot_price, volatility, flat_curve, val_date)
tree_price = trees.binomial_price(
    amer_call, spot_price, volatility, flat_curve, val_date, steps=200
)

print("--- Call Option Comparison (ATM) ---")
print_comparison("Call Option", bs_price, tree_price)

# The small difference is due to the discrete nature of the tree vs continuous BSM.

--- Call Option Comparison (ATM) ---
Call Option          | BS (Euro): 10.4506 | Tree (Amer): 10.4406 | Diff: -0.0100


## 3. Put Options: The Value of Early Exercise

For **Put Options**, early exercise *can* be optimal, especially if the option is deep In-The-Money (ITM) and interest rates are high. 

Why? Because if you exercise, you get cash ($K - S$) immediately, which you can deposit to earn interest. Waiting delays this cash receipt.

Let's price a Deep ITM Put: Spot = 60, Strike = 100.

In [3]:
deep_itm_spot = 60.0

# Define Instruments
eur_put = instruments.EuropeanOption("TEST", strike, expiry, "put")
amer_put = instruments.AmericanOption("TEST", strike, expiry, "put")

# Price
bs_put = equity.black_scholes_price(eur_put, deep_itm_spot, volatility, flat_curve, val_date)
tree_put = trees.binomial_price(
    amer_put, deep_itm_spot, volatility, flat_curve, val_date, steps=200
)

print("\n--- Put Option Comparison (Deep ITM) ---")
print(f"Intrinsic Value: {100.0 - 60.0:.4f}")
print_comparison("Put Option", bs_put, tree_put)

premium = tree_put - bs_put
print(f"\nEarly Exercise Premium: {premium:.4f}")


--- Put Option Comparison (Deep ITM) ---
Intrinsic Value: 40.0000
Put Option           | BS (Euro): 35.1774 | Tree (Amer): 40.0000 | Diff:  4.8226

Early Exercise Premium: 4.8226


**Observation**: 
* The European Put (BS) trades *below* intrinsic value (40.0) because it discounts the strike price payment.
* The American Put (Tree) trades *at* intrinsic value (40.0), because the tree logic detected that exercising immediately is better than waiting.

## 4. Tree Convergence Analysis

Binomial trees approximate the continuous log-normal distribution. As we increase the number of steps ($N$), the price oscillates and converges to the theoretical value.

Let's visualize this convergence for the European Call.

In [4]:
step_range = [10, 20, 50, 100, 200, 500]

print(f"{'Steps':<10} | {'Tree Price':>12} | {'Error':>12}")
print("-" * 42)

# Theoretical Truth (BSM)
truth = equity.black_scholes_price(eur_call, spot_price, volatility, flat_curve, val_date)

for n in step_range:
    p = trees.binomial_price(amer_call, spot_price, volatility, flat_curve, val_date, steps=n)
    err = p - truth
    print(f"{n:<10} | {p:>12.4f} | {err:>12.4f}")

Steps      |   Tree Price |        Error
------------------------------------------
10         |      10.2534 |      -0.1972
20         |      10.3513 |      -0.0993
50         |      10.4107 |      -0.0399
100        |      10.4306 |      -0.0200
200        |      10.4406 |      -0.0100
500        |      10.4466 |      -0.0040
